In [1]:
import pandas as pd
import spacy
#import spacy_transformers
import pathlib
import os
#import fr_dep_news_trf
#import openai #!pip install openai
# Il faut aller chercher le token de openai pour utiliser leur API
# L'idée est de constituer un résumé pour chacun des documents et utiliser un moteur de recherche par mot clé sur les textes ?
# Ou sinon uniquement par tag ?



In [4]:
df = pd.read_csv("../data2/data_article_clean.csv")
df

,Unnamed: 0.1,Unnamed: 0,URL,TITRE,IDinDB,TYPE,RESUME,ARTICLE,DATE,YEAR
0,311,311,https://www.millenaire3.com/dossiers/2013/ritu...,L'accompagnement des personnes en deuil : qual...,10055,Étude,Au fil des évolutions des sociétés occidentale...,Sommaire : 1 - Besoin d’information et de tran...,2013-09-30,2013
1,2465,2465,https://www.millenaire3.com/ressources/deux-mi...,Deux mille ans de culture à Lyon,10096,Texte,NaN,Lyon n’a connu dans son histoire que quelques ...,2000-01-01,2000
2,3209,3209,https://www.millenaire3.com/ressources/modelis...,Modéliser pour gérer la ville durable,10143,Texte,Article écrit pour la revue M3 n°5\n\nLa ville...,"Le projet Vélino’v, démarré en février 2013, i...",2013-06-01,2013
3,2809,2809,https://www.millenaire3.com/ressources/la-vill...,La ville comme système complexe,10184,Texte,Article écrit pour la revue M3 n°5\n\nLes scie...,"L’étude des systèmes complexes, interdisciplin...",2013-06-01,2013
4,2914,2914,https://www.millenaire3.com/ressources/le-role...,Le rôle de l’institution hospitalière dans l’u...,10240,Texte,NaN,"Anne-Sophie Clemençon, historienne de l’archit...",2002-01-01,2002
...,...,...,...,...,...,...,...,...,...,...
2114,3494,3494,https://www.millenaire3.com/ressources/travail...,"Travail, emplois, activités… Quelles mutations...",NaN,Article,Le Conseil de développement s’est saisi de ces...,L’emploi salarié à l’épreuve de trois mutation...,2016-03-01,2016
2115,3518,3518,https://www.millenaire3.com/ressources/un-nouv...,Un nouvel objectif pour les politiques de soli...,NaN,Texte,Les politiques sociales contemporaines de soli...,Les politiques sociales contemporaines de soli...,2012-09-01,2012
2116,3592,3592,https://www.millenaire3.com/ressources/vision-...,Vision 2040 du véhicule autonome,NaN,Texte,Les véhicules autonomes ne sont plus une ficti...,"Du VAL (Lille 1983), entièrement automatisé su...",2014-11-30,2014
2117,3593,3593,https://www.millenaire3.com/ressources/vitalit...,Vitalité de l’engagement associatif et du béné...,NaN,Texte,"Contrairement à certaines idées reçues, le bén...","Selon les enquêtes les plus récentes, les Fran...",2014-12-22,2014


In [10]:
def get_lemma(nlp):

    rows, cols = df.shape 

    for row in range(rows):
        txt = []
        art = str(df.iloc[row].ARTICLE)
        res = str(df.iloc[row].RESUME)
        titre = str(df.iloc[row].TITRE)
        id = df.iloc[row]["Unnamed: 0"]
        
        full = titre+'\n'+res+'\n'+art
        doc2 = nlp(res+'\n'+art)
        
        for token in doc2:
            if token.is_stop == False :#and token.pos_ != "PUNCT":
                txt.append(token.lemma_)
            
        pathlib.Path(f'./data/Texte/{id}').mkdir(parents=True, exist_ok=True)
        with open(f"./data/Texte/{id}/full.txt", "w", encoding="utf8") as f:
            f.write(full)
        with open(f"./data/Texte/{id}/texte.txt", "w", encoding="utf8") as f:
            f.write(" ".join(txt))
        
        print(f"\rSuccessfuly exported {row} / {rows} : {txt[0:10]}", end="")
    
    

In [11]:
# get_lemma(nlp_eff)

Successfuly exported 2773 / 2774 : ['objectif', '2050', ':', 'stratégie', 'national', 'carbone', '(', 'SNBC', ')', ',']nse', 'moyen', 'intégration']']ement']énagement']er']

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import glob
import pandas as pd

In [12]:
df = pd.read_csv("./data/data_article_clean.csv", index_col = 0)
fichiers = glob.glob('data/Texte/*/texte.txt')
nOcc = {}
documents=[]
urls=[]
for fichier in fichiers:
    idx = fichier.split("/")[2]
    url = df[df.index == int(idx)].URL.values[0]
    urls.append(url)
    with open(fichier, encoding='utf8') as f:
        documents.append(f.read())
counter=CountVectorizer()
nOcc=counter.fit_transform(documents)
nOcc=pd.DataFrame.sparse.from_spmatrix(nOcc,index=urls,columns=counter.get_feature_names_out())
nOcc.shape

tfidfer=TfidfVectorizer(lowercase=False) #On ne maîtrise pas bien la tokennisation de CountVectorizer !
tfidf_sklearn=tfidfer.fit_transform(documents)
tfidf_sklearn=pd.DataFrame.sparse.from_spmatrix(tfidf_sklearn,index=urls,columns=tfidfer.get_feature_names_out())




In [27]:
## To export term_doc matrices
# aut_info = pd.read_csv("./pages/data/auteurs_db.csv", index_col=0)

# for auteur in aut_info.auteurs.unique():
#     print(auteur)
#     tmp = aut_info[aut_info.auteurs == auteur].index
#     #pathlib.Path(f'./pages/data/Txt/Auteurs/{auteur}').mkdir(parents=True, exist_ok=True)
#     if len(tmp) == 1:
#         tmp2 = tfidf_sklearn.loc[[tmp[0]]].sum()
#     else:
#         tmp2 = tfidf_sklearn[tfidf_sklearn.index.isin(tmp)].sum()
#     tmp2 = tmp2[tmp2 > 0].sort_values(ascending=False)
#     tmp2.to_csv(f"./pages/data/Txt/Auteurs/TF_IDF/{auteur}.csv")


Pierre-Alain FOUR
Caroline BROSSAT
Claire HARPET
Emmanuel ARLOT
Ludovic VIÉVARD
Bertrand LABASSE
Cédric POLÈRE
Miloud LEKOUARA
Stéphane BIENVENUE
Laure BORNAREL
Marianne CHOUTEAU
Annie HONNORAT
Bruno BIGOURDAN
Sandra DECELLE
Emmanuelle LAMBREY
Paul BOINO
Abdel MOKTARI
Geneviève BERNARDIN
Brigitte YVRAY-DANGUIS
Pierre MICHEL_
Julien CASALS
Sylvie  MAURIS-DEMOURIOUX
Catherine PANASSIER
Geoffroy BING
Caroline JANUEL
Florence LE NULZEC
Catherine FORET
Emile HOOGE
Gilles CAYUELA
Valérie PUGIN
Lionel GASTINE
Stéphane AUTRAN
Jean-Loup MOLIN
Boris CHABANEL
Anne-Caroline JAMBAUD
Sylvie  BOUTAUDOU
Isabelle ROUGÉ
Quentin ZARAGORI
Nicolas NOVA
Aurélien BOUTAUD
Didier VANONI
Philippe DUJARDIN
Matthieu BOURY
Pierre GROSDEMOUGE
Pauline KERTUDO
Cécile COULMAIN
Philippe GARGOV
Clément JOURDAN
François MAYSSAL
Natacha ISRAËL
Anouk JORDAN
Frédérique SONNET
Sophie KELLER
Annabelle RICHARD
Théo BATTAGLIA
David CHEVALLIER
Benjamin LIPPENS
Eddy MAAROUFI
Catherine DEBRAND
Nicolas RIO
Benjamin  PRADEL
Julie JE

In [11]:
rand = aut_info.auteurs.unique()[3]
df = pd.read_csv(f"pages/data/Txt/Auteurs/Occurences/{rand}.csv")
df.columns = ["Terme", "Occ"]
df

,Terme,Occ
0,lyon,53
1,faire,44
2,ville,37
3,cuisine,32
4,etc,29
...,...,...
1425,hugouvieux,1
1426,huil,1
1427,hyperconcentration,1
1428,hyperurbain,1


In [32]:
#### To export tag term_doc matrices
method = "tfidf"
print(method)
if method == "tfidf":
    method = tfidf_sklearn
    export = "TF_IDF"
else:
    method = nOcc
    export = "Occurences"

tag_info = pd.read_csv("./pages/data/tags_db.csv", index_col=0)

for i, tag in enumerate(tag_info.tags.unique()):
    tmp = tag_info[tag_info.tags == tag].index
    print(tag, i, "/", len(tag_info.tags.unique()), "Nb_urls :", len(tmp))
    tmp2 = method[method.index.isin(tmp)].sum()
    tmp2 = tmp2[tmp2 > 0].sort_values(ascending=False)
    tmp2.to_csv(f"./pages/data/Txt/Tags/{export}/{tag}.csv")

tfidf
Numérique 0 / 57 Nb_urls : 205
Institution 1 / 57 Nb_urls : 147
Valeurs 2 / 57 Nb_urls : 146
Recherche 3 / 57 Nb_urls : 158
Innovation 4 / 57 Nb_urls : 238
Sécurité 5 / 57 Nb_urls : 31
Culture 6 / 57 Nb_urls : 188
Grand Lyon 7 / 57 Nb_urls : 203
